In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(['Australia'])][['year', 'countrycode', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna']]
data = data.loc[(data['year'] >= 1960) & (data['year'] <= 2000)].dropna()

# Calculate additional variables
data['alpha'] = 1 - data['labsh']  #α
data['cap_adj']=data['rkna']**data['alpha']#K^α
data['lab_adj']=data['emp']**(1-data['alpha'])#L^1-α
data['productivity']=data['rgdpna']/(data['cap_adj']*data['lab_adj'])#A
data['y_n'] = data['rgdpna'] / data['emp']  # Y/N
data['hours'] = data['emp'] * data['avh']  # L
data['tfp_term'] = data['rtfpna'] ** (1 / (1 - data['alpha']))  # A^(1/(1-alpha))
data['cap_term'] = (data['rkna'] / data['rgdpna']) ** (data['alpha'] / (1 - data['alpha']))  # (K/Y)^(alpha/(1-alpha))
data['lab_term'] = data['hours'] / data['pop']  # L/N
data= data.sort_values('year').groupby('countrycode').apply(lambda x: x.assign(
    CapitalShare=1 - x['labsh'],
    cap_adj=x['rkna']**x['alpha'],
    y_n_shifted=100 * x['y_n'] / x['y_n'].iloc[0],
    tfp_term_shifted=100 * x['tfp_term'] / x['tfp_term'].iloc[0],
    cap_term_shifted=100 * x['cap_term'] / x['cap_term'].iloc[0],
    lab_term_shifted=100 * x['lab_term'] / x['lab_term'].iloc[0]
)).reset_index(drop=True).dropna()

# Print first 10 rows of data
print(data[['countrycode', 'capitalshare', 'cap_adj', 'lab_term_shifted']].head(10))

もしかして新しいコミットをメッセージとして残せない？


